In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
%matplotlib widget
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import chemiscope
from widget_code_input import WidgetCodeInput
from ipywidgets import Textarea
from iam_utils import *
import ase
from ase.io import read, write
from ase.calculators import lj, eam
from ase.optimize import BFGS, LBFGS
import copy

In [ ]:
#### AVOID folding of output cell 

In [ ]:
%%html

<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:4000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [ ]:
data_dump = WidgetDataDumper(prefix="module_05")
display(data_dump)

In [ ]:
module_summary = Textarea("general comments on this module", layout=Layout(width="100%"))
data_dump.register_field("module-summary", module_summary, "value")
display(module_summary)

_Reference textbook / figure credits: 
Charles Kittel, "Introduction to solid-state physics", Chapters 20-21; 
[Helmut Föll, Defects in Crystals](https://www.tf.uni-kiel.de/matwis/amat/def_en/index.html) 
Wikipedia_

# Point defects

Point defects are the simplest kind of imperfections that can be found in the structure of a crystalline material. _Vacancies_ are empty lattice sites, _interstitials_ are additional atoms that fill a void in the structure.  Impurities can also occur in the form of _substitutional_ atoms, that replace some of the atoms of the main lattice. Defects can have a large impact on the electronic properties of a material, drive transport of ions, and are often related to solid-state transformations. 

Particulary for ionic crystals, where defects must preserve  overall charge neutrality, defects must come in pairs: _Frenkel defects_ are a vacancy-interstitial pair (conceptually corresponding to an atom moved from a lattice to an interstitial site), _Schottky defects_ are paris of vacancies formed by atoms with opposite charge. 

<img src="figures/defecttypes.png" width="600"/>

In the vast majority of cases the presence of a defect results in the *increase* of the energy of the crystal by an amount $\Delta E$. Then why are there defects in solids? The answer involves a combination of thermodynamics and kinetics. You may recall that the stability of a system at constant temperature $T$ and pressure is determined by its *Gibbs free energy* $G=H-TS$, where $H$ is the enthalpy (which matches to a very good approximation the internal energy for a solid at ambient pressure) minus a term that involves the *entropy* of the system. 

A full discussion of the microscopic definition of entropy is beyond the scope of this module, but we will just say that one can define it as $S=k_B \ln \Omega$, where $k_B=1.38\cdot 10^{-23}$ J/K, and $\Omega$ is the number of microscopic configurations that are consistent with the macroscopic observables and boundary conditions. For a lattice with $N$ sites and $M$ vacancies, one can estimate $\Omega$ by combinatorial arguments

$$
\ln \Omega = \ln \frac{N!}{M!(N-M)!} \approx N \ln N - M\ln M -(N-M)\ln (N-M)
$$

where we also used Stirling approximation $\ln N! \approx N \ln N$. 

By ignoring constant terms that depend only on $N$, and rewriting in terms of the vacancy concentration $x=M/N$, one can find $N^{-1}\ln\Omega \approx -x\ln x -(1-x)\ln(1-x)$. Thus, the molar free energy $G/N$ of a solid with a concentration of vacancies $x$ can be written as 

$$
g(x) =  x \Delta E  + k_B T\left[ x\ln x + (1-x)\ln(1-x)\right]
$$

<span style="color:blue">**01a** The equilibrium concentration of vacancies can be obtained differentiating the expression and solving for $x$. Write the analytical expression (you do not need to write down the steps, for this question the expression is enough). What is the highest _equilibrium_ molar vacancy concentration that is theoretically possible?</span>

In [ ]:
ex1a_txt = Textarea("Write the answer here.", layout=Layout(width="100%"))
data_dump.register_field("ex1a-answer", ex1a_txt, "value")
display(ex1a_txt)

In [ ]:
# Boltzman constant in eV/K
kB = 8.617333262145e-5

def plot_molar_free_energy(axes, deltaE, T):
    # x = M/N
    x = np.linspace(0.001, 0.999, 100)
    
    molar_free_energy_fun = lambda x: x*deltaE+kB*T*(x*np.log(x)+(1-x)*np.log(1-x))
    molar_free_energies = molar_free_energy_fun(x)
    beta = 1/(kB*T)
    equilibrium_vacancy_concentration_at_T = np.exp(-deltaE/(2*kB*T))/(2*np.cosh(deltaE*beta/2))
    molar_free_energies_at_equilibrium = molar_free_energy_fun(equilibrium_vacancy_concentration_at_T)
    
    temperatures = np.linspace(10, 4000, 80)
    equilibrium_vacancy_concentrations = np.exp(-deltaE/(2*kB*temperatures))/(2*np.cosh(deltaE/(2*kB*temperatures)) )

    axes[0].clear()
    axes[0].plot(x, molar_free_energies)
    axes[0].scatter(equilibrium_vacancy_concentration_at_T, molar_free_energies_at_equilibrium, color='red', s=80, edgecolors='black')
    axes[0].set_title('')
    axes[0].set_xlim(0.001,0.999)
    axes[0].set_ylim(-0.3,0.5)
    axes[0].set_xlabel("Vacancy concentration $x$")
    axes[0].set_ylabel("Molar free energy $g(x)$ / eV")


    axes[1].clear()
    axes[1].plot(temperatures, equilibrium_vacancy_concentrations, c='r')
    axes[1].scatter(T, equilibrium_vacancy_concentration_at_T, color='red', s=80, edgecolors='black')
    axes[1].set_title('')
    axes[1].set_xlim(0,4000)
    axes[1].set_ylim(0,0.5)
    axes[1].set_xlabel("Temperature $T$ / K")
    axes[1].set_ylabel("Equilibrium vacancy concentration")

In [ ]:
fig_ax = plt.subplots(1, 2, figsize=(8,3.8), tight_layout=True)

mfe_pb = WidgetParbox( deltaE=(0.2, 0.01, 0.3, 0.01, r"$\Delta E\, /\, eV$"), 
                       T=(2000, 300, 4000, 50, r"$T\, /\, K$ "))

ex01_wp = WidgetPlot(plot_molar_free_energy, mfe_pb, fig_ax=fig_ax)
display(ex01_wp)

The widgets above show the molar free energy of the defective crystal as a function of vacancy concentration for the temperature and vacancy formation energy selected in the sliders. The right-hand panel shows the equilibrium concentration _as a function of temperature_ (i.e. the red point on the free-energy curve determines the value of the concentration on the right-hand plot for that value of $T$. Manipulating this widget and observing the qualitative and quantitative behavior, answer the following questions.

<span style="color:blue"> **01b** What is the general trend of the equilibrium vacancy concentration with respect to temperature? How does $\Delta E$ influence this trend?</span>

In [ ]:
ex1b_txt = Textarea("Write the answer here.", layout=Layout(width="100%"))
data_dump.register_field("ex1b-answer", ex1b_txt, "value")
display(ex1b_txt)

<span style="color:blue">**02** Do you think that it would be possible, in practice, to reach the maximum equilibrium vacancy concentration? What might happen before that? </span>

In [ ]:
ex2_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex2-answer", ex2_txt, "value")
display(ex2_txt)

<span style="color:blue">**03** Metals at room temperature often have a much higher vacancy concentration than it would be predicted by the expressions above. How can you explain this observation? </span>

In [ ]:
ex3_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex3-answer", ex3_txt, "value")
display(ex3_txt)

# Vacancy formation energy

In order to compute the vacancy formation energy, we need first to compute the energy of an atom in a perfect crystal. If the calculation is performed for a supercell with $n$ atoms, the molar energy is the total potential for the cell divided by $n$, $\epsilon = E_n / n$. 


One then needs to build a supercell (that could contain a different number $m$ of atoms) and remove one. The energy of a vacancy can be computed by subtracting from the energy of the supercell that of $m-1$ atoms of a perfect crystal, 

$$ 
\Delta E = E_{m-1} - (m-1)\epsilon
$$

The visualizer below shows an _fcc_ structure, with one vacancy. Can you see what atom is missing? The second frame in the viewer shows green points at the ideal _fcc_ lattice positions, to help you identify the vacancy. 

In [ ]:
from ase.lattice.cubic import FaceCenteredCubic
import copy

al_fcc = FaceCenteredCubic(size=(2,2,2), symbol='Al', pbc=True)
ref_f = al_fcc.copy(); ref_f.symbols = "F"*len(ref_f)
al_fcc_vacancy = copy.deepcopy(al_fcc)

del al_fcc_vacancy[17]
al_fcc_strucs = [al_fcc, al_fcc_vacancy]

cs_vacancy = chemiscope.show([al_fcc_vacancy, al_fcc_vacancy+ref_f], mode="structure",                      
                     settings={"structure":[{"bonds":False, "unitCell":True,"supercell":{"0":1,"1":1,"2":1},
                                            "environments": {"cutoff": 40}}]}                    
                    )
display(cs_vacancy)

<span style="color:blue">**04** Write a function that computes the vacancy formation energy of aluminum (EAM potential), using a conventional _fcc_ cell (4-atoms, lattice parameter $a_0 = 4.05$Å) to get the molar energy, and a supercell replicated _nrep_ times in each direction, from which you can then remove one atom. Return the vacancy formation energy computed from the formula above. </span>

_NB: you can remove an atom from an `ase.Atoms` structure by calling `del structure[[index]]` (note the double brackets). See the [module on interatomic potentials](./04-Potentials.ipynb) to remind you how the ASE potential calculator works_

In [ ]:
ex04_wci = WidgetCodeInput(
        function_name="vacancy_builder", 
        function_parameters="nrep",
        docstring="""
Builds a model with a nrep×nrep×nrep supercell and a vacancy,
and returns the vacancy formation energy.

:param nrep: number of repetitions for the supercell
:return e_vacancy: vacancy formation energy
""",
        function_body="""
import numpy as np
from ase import Atoms
from ase.calculators import eam

return 0 # <-- remove this line after having finished the implementation of the function
calc =  ...  # initializes the calculator 

## constructs an fcc unit cell
a0 = 4.05 # lattice parameter of Al
# constructs a unit cell
fcc_cell = Atoms("Al4", cell=..., positions=..., pbc=True, calculator=calc ) 
# and computes the energy
e_cell = ...

## creates the vacancy supercell
vacancy = fcc_cell.replicate( ... )
vacancy.calc = calc
# removes an atom
...
e_supercell = ...

# compute formation energy combining e_cell and e_supercell
e_vacancy = ...
# print("nrep = %d,  E_vac = %.4f eV" %(nrep, e_vacancy)) # <- if you want to see the precise value printed out, uncomment this line
return e_vacancy
"""
        )

def plot_vac(ax):
    ex04_out.clear_output()
    func = ex04_wci.get_function_object()
    with ex04_out:
        values = np.asarray([ [n, func(n)] for n in [1,2,3,4]])
    
    ax.plot(values[:, 0], values[:,1], 'b*')
    
    ax.set_xlabel(r"$n_{\mathrm{rep}}$")
    ax.set_ylabel(r"$E_{\mathrm{vac}}$ / eV")
    #ax.set_ylim([-3, 3])

    
ex04_out = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))
ex04_plot = WidgetPlot(plot_vac)

data_dump.register_field("ex4-function", ex04_wci, "function_body")        
ex04_wcc = WidgetCodeCheck(ex04_wci, ref_values = {(1,) : 1.0199205493799237, (2,) : 0.770153142090478},
                           demo=(ex04_plot, ex04_out))
display(ex04_wcc)

<span style="color:blue">**05** What is the vacancy formation energy? Does the estimated value change with cell size? Does it depend on the index of the atom that is removed? Comment on what you observe. </span>

In [ ]:
ex5_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex5-answer", ex5_txt, "value")
display(ex5_txt)

# Energy relaxation

The exercise above assumes that atoms in the vicinity of a vacancy remain in their ideal lattice position. This is obviously a harsh approximation: with the change in environment, atoms will rearrange to find a more favorable position. This will both change the structure, and the energy of the system, and therefore affect the vacancy formation energy. 

To account for this rearrangement, one can _relax_ the geometry of the vacancy structure. The idea is simply to minimize the potential energy $V$ of the system, looking for a structure for which the force $\mathbf{f}=-\nabla V$ is zero. There is a large number of schemes that have been proposed to achieve this goal (see here those that are [implemented in ASE](https://wiki.fysik.dtu.dk/ase/ase/optimize.html)) which underscores the importance of energy relaxation to compute accurately the stability of structures. 

The most naïve approach, called [steepest descent](https://en.wikipedia.org/wiki/Gradient_descent) iteratively optimizes the structure following the gradient, 

$$
\mathbf{r} \leftarrow \mathbf{r} -\alpha \nabla V(\mathbf{r}).
$$

Even though it may seem to be an efficient idea (following the direction of maximum decrease of the potential) in a high-dimensional problem it leads to very slow convergence. If you are curious, you can read about the mathematical aspects of optimization, and why steepest descent does not work, in these excellent [lecture notes](https://www.cs.cmu.edu/~quake-papers/painless-conjugate-gradient.pdf). 

Here we will use a rather sophisticated method, named [BFGS](https://en.wikipedia.org/wiki/Broyden%E2%80%93Fletcher%E2%80%93Goldfarb%E2%80%93Shanno_algorithm) from the names of its inventors. 
We sometimes will use a version of BFGS with reduced computer memory usage called [LBFGS](https://en.wikipedia.org/wiki/Limited-memory_BFGS).
In practice, a BFGS/LBFGS optimization can be run as follows:

```python
from ase.optimize import BFGS, LBFGS
structure = Atoms( ... , calculator=...)   # you can also initialize the structure and set the calculator with structure.calc = ...
opt = BFGS(structure) # LBFGS(structure)
opt.run(fmax=0.01)
# the optimized geometry is stored in the `structure` object
print(structure.positions, structure.get_potential_energy())
```


We will start by looking at a pre-coded function that minimizes the energy of a Lennard-Jones dimer.

_NB: To trigger a re-calculation, you must click the update button after changing the sliders_

In [ ]:
ex06_wci = WidgetCodeInput(
        function_name="optimize_lj_dimer", 
        function_parameters="sigma, epsilon, fmax",
        docstring="""
Optimizes the geometry He dimers using a LJ potential.

:param sigma: LJ sigma
:param epsilon: LJ epsilon
:param fmax: threshold to reach so that the optimizer stops

:return filename: filename of the optimization trajector of the structure
""",
        function_body="""
import ase 
from ase.optimize import BFGS
from ase.calculators import lj

ljcalc = lj.LennardJones(sigma=sigma, epsilon=epsilon, rc=4*sigma)
structure = ase.Atoms('He2',
                      positions=[[-2,0,0],[2,0,0]],
                      pbc=False)
structure.calc = ljcalc
filename='lj_dimer_opt.xyz'
opt = BFGS(structure, trajectory=filename)
opt.max_steps=100
opt.run(fmax=fmax)
# returns just the filename, which will be used by the visualizer
return filename
"""
)

In [ ]:
    
def fun_ex06(change={'type':'change'}):
    ex06_out.clear_output()
    fun_inputs = copy.deepcopy(ex06_pb.value)
    fun_inputs['fmax'] = 10**ex06_pb.value['log10fmax']
    fun_inputs.pop('log10fmax')
    with ex06_out:
        fname = ex06_wci.get_function_object()(**fun_inputs)
    frames = ase.io.read(fname, ':')
    
    energies = np.asarray([f.calc.results['energy'] for f in frames])
    
    properties=dict(
                index=np.arange(len(frames)),
                energy=energies,
                distances=[f.get_distance(0,1) for f in frames],
                )

    with ex06_up:
        display(chemiscope.show(frames, properties, mode="structure",
                         settings={"structure":[{"bonds":False, "unitCell":False,"supercell":{"0":1,"1":1,"2":1},
                                                "environments": {"cutoff": 40}}]})
               )
    
ex06_up = WidgetUpdater(updater=fun_ex06)
ex06_out = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))

ex06_pb = WidgetParbox(sigma=(2.6, 0.1, 5., 0.1, r"$\sigma$"), 
                        epsilon=(0.5, 0.1, 0.7, 0.01, r"$\epsilon$"),
                        log10fmax=(-3, -5, 0, 1, r"$\log_{10}(\mathbf{f}_\textrm{max})$"))
ex06_wcc = WidgetCodeCheck(ex06_wci, ref_values = {}, demo = (ex06_pb, ex06_out, ex06_up))

display(ex06_wcc)

<span style="color:blue">**06a** Read the function above, that takes as arguments the parameters of a LJ potential, initializes a dimer at separation equal to $\sigma$, and runs a geometry optimization.  Observe in the visualizer the behavior of the energy and distance. Are the equilibrium separation and binding energy compatible with the analytical result for the LJ dimer (see module 4)?</span>

In [ ]:
ex6a_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex6a-answer", ex6a_txt, "value")
display(ex6a_txt)

<span style="color:blue">**06b** How many steps are needed to reach convergence (set $\sigma=2.6, \epsilon=0.5, \mathbf{f}_\textrm{max}=0.001$ to ensure reproducibility) ? How many steps are needed if you make the convergence threshold 10 times larger and smaller? </span>

In [ ]:
ex6b_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex6b-answer", ex6b_txt, "value")
display(ex6b_txt)

<span style="color:blue">**07a** Combine the code for the vacancy formation energy calculation with that for the geometry optimization of the LJ dimer, to write a function that relaxes the vacancy geometry and returns the vacancy formation energy. </span>

_NB: use the LBFGS optimizer, it's more memory efficient_

In [ ]:
ex07_wci = WidgetCodeInput(
        function_name="vacancy_relax", 
        function_parameters="nrep",
        docstring="""
Builds a model with a nrep×nrep×nrep supercell and a vacancy, relaxes the geometry,
and returns the vacancy formation energy.

:param nrep: number of times the fcc unit cell should be replicated in building the supercell

:return e_vacancy: vacancy formation energy
""",
        function_body="""
import numpy as np
from ase import Atoms
from ase.optimize import LBFGS
from ase.calculators import eam

return 0, None, None # <-- remove this after having completed the code below
calc =  ...  # initializes the calculator 

## constructs an fcc unit cell
a0 = 4.05 # lattice parameter of Al
# constructs a unit cell
fcc_cell = Atoms("Al4", cell=..., positions=..., pbc=True, calculator=calc ) 
# and computes the energy
e_cell = ...

## creates the vacancy supercell
vacancy = fcc_cell.repeat( ... )
vacancy.calc = calc
# removes an atom
...

## runs the optimization here (don't change the file name if you want to see the trajectory)
filename
opt = LBFGS

e_supercell = ...

# compute formation energy combining e_cell and e_supercell
e_vacancy = ...
# print("nrep = %d,  E_vac = %.4f eV" %(nrep, e_vacancy)) # <- if you want to see the precise value printed out, uncomment this line
return e_vacancy, fcc_cell, filename
"""
        )
data_dump.register_field("ex7-function", ex07_wci, "function_body")

In [ ]:
def fun_ex07(change={'type':'change'}):
    ex07_out.clear_output()
    with ex07_out:
        evac, fname, cell = ex07_wci.get_function_object()(**ex07_pb.value)
    if fname is None: 
        return
    frames = ase.io.read(fname, ':')
    nrep = ex07_pb.value["nrep"]
    suxcell = cell.repeat(nrep)
    suxcell.symbols = "F"*len(suxcell)    

    energies = np.asarray([f.calc.results['energy'] for f in frames])
    for frame in frames:
        frame += suxcell
        
    properties=dict(
                index=np.arange(len(frames)),
                energy=energies,
                )

    with ex07_up:
        display(chemiscope.show(frames, properties, mode="structure"
                         settings={"structure":[{"bonds":False, "unitCell":False,"supercell":{"0":1,"1":1,"2":1},
                                                "environments": {"cutoff": 40}}]})
               )


ex07_up = WidgetUpdater(updater=fun_ex07)
ex07_out = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))

ex07_pb = WidgetParbox(nrep=(2,1,4,1, r"$n_{\mathrm{rep}}$"))
ex07_wcc = WidgetCodeCheck(ex07_wci, ref_values = {}, demo = (ex07_pb, ex07_out, ex07_up))

display(ex07_wcc)

After you finish the function above, you can plot the convergence of $E_{\mathrm{vac}}$ with the supercell size. The computation might take several minutes.

In [ ]:
def plot_vac(ax):
    vacout.clear_output()
    # Somehow the ax is not cleared in the widget on the first run
    # rerunning the cell works, but weirdly the computation also
    # immediately starts
    # todo for next time
    ax.clear()
    #ax.get_figure().canvas.flush_events()
    func = ex07_wci.get_function_object()
    with vacout:
        print("Computing convergence curve")
        values = np.asarray([ [n, func(n)[0]] for n in [1,2,3,4]])
    ax.plot(values[:, 0], values[:,1], 'b*')
    
    ax.set_xlabel(r"$n_{\mathrm{rep}}$")
    ax.set_ylabel(r"$E_{\mathrm{vac}}$ / eV")

vacout = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))
vacbutton = Button(description="Compute curve")
vacbox = VBox([vacbutton, vacout])
display(vacbox)
vacplot = WidgetPlot(plot_vac)
vacbutton.on_click(vacplot.update)
vacbox.children += (vacplot,)

<span style="color:blue">**07b** What is the vacancy formation energy, after relaxation (take a cell size with 4  repetitions as the converged value)? Compare the system-size convergence with what you observed in the unrelaxed case. What changed? Why the difference? </span>

In [ ]:
ex7b_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex7b-answer", ex7b_txt, "value")
display(ex7b_txt)

# Decohesion curves

The surface energy of a material $\gamma$ is the energy that one has to pay to create a unit area of surface. It is an important parameter that has implications for fracture mechanics, tribology, and adhesion. 

<img src="figures/surface-energy-2.png" width="600"/>

In a computational model, one can realize quite easily the thought experiment of achieving a clear-cut cleavage of a crystal along a high-symmetry surface, using exactly the same strategy discussed in [the crystallography module](./02-Crystallography.ipynb) to generate a surface: a bulk structure is formed with a unit cell aligned along appropriate directions, and then one of the lattice parameter is extended, to create a slab geometry, with _two_ surfaces separated by vacuum. 

By computing the energy as a function of separation, we can build _decohesion curves_ that give a cue on the magnitude of the surface energy. The curve is usually expressed such that the energy for the ideal crystal structure is zero, and the energy at large separation corresponds to the surface energy. 

In this exercise and the next we will need a supercell with the axes aligned along different high-symmetry lattice directions, namely $(111)$, $(1\bar{1}0)$, $(11\bar{2})$. This was an optional task in [exercise 7, module 2](./02-Crystallography.ipynb), and here you can see the (non-trivial) solution. It is easy to see that those three directions are mutually orthogonal (which is useful as the resulting unit cell is orthorhombic), less easy to see how the bizarre coordinates generate a basis for an _fcc_ structure. Visualize the structure by clicking the `update` button, and try to replicate the cell along different directions to familiarize yourself with the different surfaces and lattice planes.

In [ ]:
fcc111_wci = WidgetCodeInput(
        function_name="fccAl111_surface", 
        function_parameters="",
        docstring="""
Returns a fcc Al (111) surface.
:return struc: fcc Al (111) surface
""",
        function_body="""
import numpy as np
from ase import Atoms
a0 = 4.05 # lattice parameter of Al
h0 = np.sqrt(np.asarray([[3,0,0],[0, 1/2, 0],[0,0,3/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],
						   [0,1/8, 9/24],
						   [1/3,0,16/24],
						   [1/3,1/8,1/24],
						   [4/3,0,4/24],
						   [4/3,1/8,25/24]]))
struc = Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True)
return struc
"""
)

fcc111_vbox = VBox([],layout=Layout(overflow='hidden'))

def fcc111_updater():
    fcc111_vbox.children = ()
    
    fcc111 = fcc111_wci.get_function_object()()

    fcc111cs = chemiscope.show(frames=[fcc111], mode="structure",
                             settings= {'structure' : [{'bonds': False,
                                                        'unitCell': True,
                                                        'spaceFilling': False,
                                                        'supercell':{'0':1,'1':1,'2':1}, 
                                                        'keepOrientation': True,}]
                                  })
    fcc111_vbox.children = (fcc111cs,)
    display(fcc111cs)

fcc111_wcc = WidgetCodeCheck(fcc111_wci, ref_values = {}, demo=WidgetUpdater(updater=fcc111_updater))
display(fcc111_wcc)

Having an orthorhombic cell aligned with the high-symmetry directions makes it easy. Below you can see an already working function that separates two surfaces from each other with increasing distance, then computes the energy and finally plots the decohesion curve. Your exercise will be in the next widget. You will quickly realize that the function computing the surface energy is only returning zero until you implement it properly!

In [ ]:
ex08_wci = WidgetCodeInput(
        function_name="surface_energy",
        function_parameters="surface, surface_energy, fccAl111_surface",
        docstring="""
Computes the surface energy for a fcc Al (111) structure.

:param surface: the direction of the surface normal, as a string ["(111)", "(1-10)", "(11-2)"]
:param surface_energy: the function you implement in the exercise 8a
:param fccAl111_surface: the function above computing a fcc Al (111) surface

:return distances: distances between surfaces
:return e_surfaces: surface energies
:return frames: the structures with separated surfaces
""",
        function_body="""
from ase.calculators import lj
import numpy as np

if surface == "(111)":
    surfac_vec_direction1 = 1
    surfac_vec_direction2 = 2
    surface_normal_direction = 0
elif surface == "(1-10)":
    surfac_vec_direction1 = 0
    surfac_vec_direction2 = 2
    surface_normal_direction = 1
elif surface == "(11-2)":
    surfac_vec_direction1 = 0
    surfac_vec_direction2 = 1
    surface_normal_direction = 2
else:
    ## We use a f-strings in the error message
    raise ValueError(f"Unknown option {surface}")

ljcalc = lj.LennardJones(sigma=2.6, epsilon=0.5, rc=4*2.6)

## builds (2,2,2)-supercell, the surface normal is later repeated
supercell_nrep = [2, 2, 2]
supercell_nrep[surface_normal_direction] = 1
fccAl111_supercell = fccAl111_surface().repeat(supercell_nrep)
fccAl111_supercell.calc = ljcalc

## Computes surface normal, it is important that we the primitive cell vector.
## Otherwise we would change the surface normal when altering the primitive
## cell vector, since they would share the same memory address
surface_vec = fccAl111_supercell.cell[surface_normal_direction].copy()
surface_normal = surface_vec/np.linalg.norm(surface_vec)

## distances for the surface separation
distances = np.linspace(0, 8, 10) # <-- 10 linearly-spaced values in range [0, 8]
e_surfaces = []
frames = []

surface_vec1 = fccAl111_supercell.cell[surfac_vec_direction1]
surface_vec2 = fccAl111_supercell.cell[surfac_vec_direction2]

nrep = [1, 1, 1]
nrep[surface_normal_direction] = 2
for distance in distances:
    ## The distance between the surfaces is increased by scaling the primitive cell vector
    ## in the direction of the surface normal, then repeating the structure in the same
    # direction
    fccAl111_supercell.cell[surface_normal_direction] = surface_vec + surface_normal*distance
    fccAl111_struc = fccAl111_supercell.repeat(nrep)
    ## Rescales the primitive cell vector to match the periodic boundary conditions on both
    ## sides of the cut.
    ## You can try to comment out this line to see what happens.
    fccAl111_struc.cell[surface_normal_direction] = 2*surface_vec + surface_normal*distance
    fccAl111_struc.calc = ljcalc

    energy = fccAl111_struc.get_potential_energy()
    ## here ex08a-function computing the surface energy is used
    e_surface = surface_energy(energy, surface_vec1, surface_vec2)
    e_surfaces.append( e_surface )
    frames.append( fccAl111_struc )
    
e_surfaces = np.asarray(e_surfaces)
## removing baseline energy at distance 0 from all energies
e_surfaces -= e_surfaces[0]
return distances, e_surfaces, frames
"""
)

In [ ]:
ex08_vb = VBox(layout=Layout())

def fun_ex08(change={'type':'change'}):
    ex08_out.clear_output()
        
    surface = ex08_pb.value['surface']
    surface_energy_fun = ex08a_wci.get_function_object()
    fcc111_surface_fun = fcc111_wci.get_function_object()
    distances, energies, frames = ex08_wci.get_function_object()(surface,
                                                             surface_energy_fun,
                                                             fcc111_surface_fun)

    properties=dict(
                distances=distances,
                surface_energy=energies,
                )

    with ex08_up:
        display(chemiscope.show(frames, properties, mode="structure",
                         settings={"structure":[{"bonds":False, "unitCell":False,"supercell":{"0":1,"1":1,"2":1},
                                                "environments": {"cutoff": 40}}]}))

ex08_up = WidgetUpdater(updater=fun_ex08)
ex08_out = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))

ex08_pb = WidgetParbox(surface=("(111)", ["(111)", "(1-10)", "(11-2)"], r"Surface normal"))
ex08_wcc = WidgetCodeCheck(ex08_wci, ref_values={}, demo = (ex08_pb, ex08_out, ex08_up))
display(ex08_wcc)


<span style="color:blue">**08a** Modify the function below so that it computes the surface energy $\gamma$ in eV/Å².</span>

In [ ]:
ex08a_wci = WidgetCodeInput(
        function_name="surface_energy",
        function_parameters="energy, surface_vec1, surface_vec2",
        docstring="""
Computes the surface energy for a structure.

:param energy: the energy of the structure with the separated surface 
:param surface_vec1: first vector spanning the separated surface
:param surface_vec2: second vector spanning the separated surface

:return e_surface: surface energy
""",
        function_body="""
from numpy import cross
from numpy.linalg import norm

# Compute these quantities
area = 1
e_surface = 0 / area

return e_surface
"""
)

In [ ]:
ex08a_vb = VBox(layout=Layout())
    
data_dump.register_field("ex8a-function", ex08a_wci, "function_body")


a0 = 4.05
h0 = np.sqrt(np.asarray([[3,0,0],[0, 1/2, 0],[0,0,3/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],[0,1/8, 9/24],[1/3,0,16/24],[1/3,1/8,1/24],[4/3,0,4/24],[4/3,1/8,25/24]]))
struc = ase.Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True )
from ase.calculators import lj
ljcalc = lj.LennardJones(sigma=2.6, epsilon=0.5, rc=4*2.6)
struc.calc = ljcalc # assigns the calculator
energy = struc.get_potential_energy()

ex08a_wcc = WidgetCodeCheck(ex08a_wci, ref_values =
                            {(energy, tuple(struc.cell[0].tolist()), tuple(struc.cell[1].tolist())): -1.2396836759068275})
display(ex08a_wcc)


<span style="color:blue">**08b** What is the surface energy of aluminum along the $(111)$, $(1\bar{1}0)$, $(11\bar{2})$ directions? </span>

In [ ]:
ex8_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex8-answer", ex8_txt, "value")
display(ex8_txt)

# Dislocations

Dislocations are line defects that can be understood as a loss of register between lattice planes - essentially one section of a plane of atoms is removed, and the remaining pieces are glued together. 
Dislocations can move when a material is subject to strain, and are involved in the mechanism that underlie plastic deformations in metals. The mechanisms by which dislocations are created and move in a real material can be [quite complicated](https://en.wikipedia.org/wiki/Frank%E2%80%93Read_source).  

<img src="figures/dislocation-edge.png" width="400"/>
<div style='text-align:center; font-style:italic'>
   Credits: Wikipedia, user Magasjukur2, License: CC-BY-SA
</div>


Even considering the idealized scenario in which a lattice plane is removed to form an edge dislocation, the image above is highly unrealistic. Atoms around a dislocation relax to form a distorted structure (_dislocation core_) surrounded by a long-range elastic field, whose form can be evaluated analytically based on continuum elasticity theory. 
A relatively simple example of the complex atomic rearrangements that can occur around a dislocation core is the splitting of a $1/2(1\bar{1}0)\{111\}$ dislocations in an _fcc_ crystal into two _partials_ separated by a stacking fault. The process is illustrated schematically below: a dislocation along the $(1\bar{1}0)$ directions corresponds to removal of _two_ $(110)$ lattice planes. The large deformation of the lattice can be accommodated by forming a pair of partial dislocations (that correspond to a single missing plane each) that are separated by a stacking fault region.

The process is shown schematically in the figure below, and we are going to investigate it with an explicit simulation

<img src="figures/split_disl_perspective.png" width="600"/>
<div style='text-align:center; font-style:italic'>
   Credits: Helmut Föll, Defects in Crystals
</div>

The function `relax_dislocation` below generates an Al _fcc_ structure. It removes a couple of $(110)$ half-planes, generating _a pair_ of dislocations at the two ends of the removed planes. It's easy to convince yourself that due to supercell periodicity it is not possible to generate a single dislocation. A distortion is then introduced to "nudge" the atoms towards closing the gap. A geometry optimization is then run to minimize the energy of the system. The demo widget shows the process, and the energies as computed by the chosen potential. 

_NB: That there are more refined schemes to build dislocation geometries, but this should give you the gist of the process._

_Don't worry if you don't understand some of the Python/numpy syntax used below. Try to focus on the "computational experiment" being performed. The function returns the pieces that are then passed to your line energy function_

In [ ]:
dislocation_wci = WidgetCodeInput(
        function_name="relax_dislocation", 
        function_parameters="line_energy",
        docstring="""
Builds and relaxes a dislocation model for a 1/2(1̅10){111} dislocation

:param line_energy: a function that computes the line energy

:return supercell: the supercell of the bulk without dislocations
:return dislocated_supercell: the supercell containing the pair of dislocations 
The supercell with perfect bulk structure, and that with the initial dislocation
""",
        function_body="""
import numpy as np
import ase
from ase.io import read
from ase.calculators import lj, eam
from ase.optimize import LBFGSLineSearch

# a LJ potential fitted to match some of the properties of FCC aluminum
calc = lj.LennardJones(sigma=2.62, epsilon=0.41, rc=2*2.62)

# creates an fcc unit cell with orientations along (111) (1̅10) (11̅2)
a0 = 4.05  # lattice parameter of Al
h0 = np.sqrt(np.asarray([[3,0,0],[0, 1/2, 0],[0,0,3/2]]))
pos0 = np.sqrt(np.asarray([[ 0, 0, 0],
                           [0,1/8, 9/24],
                           [1/3,0,16/24],
                           [1/3,1/8,1/24],
                           [4/3,0,4/24],
                           [4/3,1/8,25/24]]))
al_bulk = ase.Atoms("Al6", cell=a0*h0, positions=pos0*a0, pbc=True )
al_bulk.calc = calc # assigns the calculator

al_bulk_energy = al_bulk.get_potential_energy() # <-- gets energy of a perfect unit cell
al_bulk_atom_energy = al_bulk_energy/len(al_bulk)

# creates a supercell (you could try larger supercells!)
supercell = al_bulk.repeat((6,24,1))  
supercell.calc = calc
dislocated_supercell = supercell.copy()

### removes a slice of atoms (two (110) half-planes), creating the dislocation
# selects the atoms in the half-planes based on their positions
sel = np.where((np.abs(supercell.positions[:,1]-supercell.cell[1,1]/2-a0*0.12) < a0*0.35+ 1e-5) &
               (np.abs(supercell.positions[:,0]-supercell.cell[0,0]/2)<supercell.cell[0,0]/4+1e-3)
               )[0]
# removes them by that generating dislocation
del(dislocated_supercell[sel]) 
dislocated_supercell.calc = calc

### pulls atoms closer at the center of the gap, to facilitate convergence 
# no need to worry about this horrible expression, just look at its effect in the visualizer!
relaxed_dislocated_supercell = dislocated_supercell.copy()
relaxed_dislocated_supercell.positions[:,1] -= (0.25*a0*np.sign(relaxed_dislocated_supercell.positions[:,1]-supercell.cell[1,1]/2)*np.exp(-(relaxed_dislocated_supercell.positions[:,1]-supercell.cell[1,1]/2)**2*0.5/(2*a0)**2)*np.exp(-(relaxed_dislocated_supercell.positions[:,0] -supercell.cell[0,0]/2)**2*0.5/((supercell.cell[0,0]/8)**2)))     

### runs geometry optimization. this will output the trajectory data 
relaxed_dislocated_supercell.calc = calc
opt = LBFGSLineSearch(relaxed_dislocated_supercell, trajectory='dislocation-lj.xyz', memory=50)
opt.run(fmax=0.001)

line_energy = line_energy(relaxed_dislocated_supercell.get_potential_energy(),
                          len(relaxed_dislocated_supercell), 
                          al_bulk_atom_energy, supercell.cell[2,2])

# this also returns a couple of things we need to plot a nice movie...
return line_energy, supercell, dislocated_supercell
"""
)

def dislocation_updater():
    stdout = Output(layout=Layout(width='100%', height='100%', max_height='200px', overflow_y='scroll'))
    vbox = VBox([stdout],layout=Layout(overflow='hidden'))
    display(vbox)
    with stdout:
        line_energy, supercell, dislocation = dislocation_wci.get_function_object()(ex09a_wci.get_function_object())
        print("Dislocation line energy: %.6f eV/Å" % (line_energy))
        
    # reads data from the geop log
    opt_traj = read('dislocation-lj.xyz',':')[::10]
    # energy of the "hard cut"
    edis = dislocation.get_potential_energy()
    
    # this accumulates the raw energies from the run. 
    energies = []
    for frame in [dislocation]+opt_traj:
        energies.append(ex09a_wci.get_function_object()(frame.get_potential_energy(), len(frame),
                                                       supercell.get_potential_energy()/len(supercell), frame.cell[2,2]))
    
    # uses dummy F atoms to show the lattice positions in the absence of the dislocation (ideal structure)
    supercell.symbols = "F"*len(supercell)
    frames=[dislocation+supercell]+[(f+supercell) for f in opt_traj]

    properties=dict(index=np.arange(1+len(opt_traj)), energy=energies)              
        
    cs = chemiscope.show(frames = frames, properties=properties,
                             settings= {'structure' : [{'bonds': False, 'unitCell': True, 'spaceFilling': False, 'supercell':{'0':1,'1':1,'2':1}, 
                                                           'keepOrientation': True,}] },
                         mode="structure"
                           )
    vbox.children += (cs,)
    

dislocation_wcc = WidgetCodeCheck(dislocation_wci, ref_values = {},
                           demo=WidgetUpdater(updater=dislocation_updater))
display(dislocation_wcc)

<span style="color:blue">**09a** The function `relax_dislocation` above is already working (and quite complex). After having understood what it does, as a minimal personal contribution, write a function that computes the _line_ energy of the dislocation given the energy and size of the dislocation structure, a reference bulk energy and the lenght of the cell in the direction parallel to the dislocation (energies are in eV, lengths in Å). Then run again the optimization of the dislocation: the energies will be plotted in terms of line energies. </span>

In [ ]:
ex09a_wci = WidgetCodeInput(
        function_name="line_energy", 
        function_parameters="dislocated_supercell_energy, dislocated_supercell_size, bulk_atom_energy, dislocation_length",
        docstring="""
Computes the dislocation line energy

:param dislocated_supercell_energy: Energy of the supercell containing the pair of dislocations
:param dislocated_supercell_size: Number of atoms in the supercell containing the pair of dislocations
:param bulk_atom_energy: Per-atom energy of the perfect bulk (total energy divided by number of atoms)
:param dislocation length: The length of the dislocation.

:return line_energy: dislocation line energy
""",
function_body="""
## You need to modify this line, it's currently just returning the raw energy
line_energy = dislocated_supercell_energy

return line_energy
""")
data_dump.register_field("ex9a-function", ex09a_wci, "function_body")        
ex09a_wcc = WidgetCodeCheck(ex09a_wci, ref_values={(123,44,0.51,0.3):167.6})
display(ex09a_wcc)

<span style="color:blue">**09b** What is the line energy for the dissociated dislocation? How many $(110)$ lattice planes separate the two partials?</span>

_Hint: remember you can look at the properties of the structure by clicking on the info panel below the molecular geometry. As for the separation between the partials, give just a rough estimate by looking at the figure: this is how the center of a partial dislocation looks like:_
<img src="figures/partial.png"  width="200"/>

In [ ]:
ex9b_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex9b-answer", ex9b_txt, "value")
display(ex9b_txt)

<span style="color:blue">**09c** (optional) Modify the function to use the Al EAM potential rather than the LJ one. What is the dislocation line energy? Observe the separation between the partials. How many lattice planes separate them? Compare with the case of the LJ model. </span>

_The EAM is much slower, launch the calculation and go get a coffee... or several!_

In [ ]:
ex9c_txt = Textarea("Write the answer here", layout=Layout(width="100%"))
data_dump.register_field("ex9c-answer", ex9c_txt, "value")
display(ex9c_txt)